In [1]:
from collections import defaultdict as ddict
from __future__ import division
import numpy as np
import h5py
import os
import matplotlib.pyplot as plt
import pprint
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [10]:
class Session:
    
    def __init__(self, data):
        self.data = data
        self.trialsdataset = self.data['Trials']
        self.trials = self.trialsdataset['trialNumber']
        self.numtrials = len(self.trials)
        self.results = self.trialsdataset['result']
        self.proportion_correct = 0
        self.hits = 0
        self.misses = 0
        self.rewards = []
        self.pc = self.calculate_percent_correct()
        
        self.flows0 = self.trialsdataset['olfas:olfa_0:mfc_1_flow']
        self.flows1 = self.trialsdataset['olfas:olfa_1:mfc_1_flow']
        
        self.odors0 = self.trialsdataset['olfas:olfa_0:odor']
        self.odors1 = self.trialsdataset['olfas:olfa_1:odor']
        
#         try:
#             self.odors0 = self.trialsdataset['olfas:olfa_0:odor']
#             self.odors1 = self.trialsdataset['olfas:olfa_1:odor']
#         except AttributeError:
#             pass
    
        self.context = self.trialsdataset['context']
        
        self.concmatdict = {1:{}, 2:{}}
        
        for key in self.concmatdict.keys():
            carvonedict = {"None":{}, '(+)-carvone':{}, '(-)-carvone':{}}
            for carkey in carvonedict.keys():
                limonenedict = {"None":[], '(+)-limonene':[], '(-)-limonene':[]}
                carvonedict[carkey] = limonenedict
            self.concmatdict[key] = carvonedict
                    
       
        print 'dictionary initialization done'
        return

        
    def calculate_percent_correct(self):
        for i in xrange(len(self.results)):
            if self.results[i] == 1 or self.results[i]==2:
                self.rewards.append(1)
                self.hits +=1
            else:
                self.rewards.append(0)
            if self.results[i] > 4:
                self.misses +=1
            
        self.proportion_correct = self.hits/(self.numtrials-self.misses)
        self.pc = 100*self.proportion_correct
        return self.pc
    
#     def make_mixture_array(self):
#         for trial in range(self.numtrials):
            

    def make_concentration_bins(self):
        matrix_list = []
        carvoneflows = [0, 10, 30, 100]
        limoneneflows = [0, 10, 30, 100]
        contextlist = [1, 2]
        carvonelist = ['(+)-carvone', 'None', '(-)-carvone']
        limonenelist = ['(+)-limonene', 'None', '(-)-limonene']

#         carvoneflows = [30]
#         limoneneflows = [30]
#         contextlist = [2]
#         carvonelist = ['(+)-carvone']
#         limonenelist = ['(+)-limonene']
        
        for trial in range(self.numtrials):
            if self.odors0[trial] == 'None':
                self.flows0[trial] = 0
            if self.odors1[trial] == 'None':
                self.flows1[trial] = 0
        counter = 0        
        for cont, key in zip(contextlist, self.concmatdict.keys()):
            
            
            for carv, carvkey in zip(carvonelist, self.concmatdict[key].keys()):
                
                for lim, limkey in zip(limonenelist, self.concmatdict[key][carvkey].keys()):
                    
                    counter += 1
                    concmat = np.zeros((4, 4, 4)) # first dimension is hits, second num trials, third percent correct, fourth misses
                    
                    for i in range(len(carvoneflows)):
                        carcon = carvoneflows[i]
                        for j in range(len(limoneneflows)):
                            limcon = limoneneflows[j]
                            
                            for trial in range(self.numtrials):
                                
                                if self.context[trial] == cont and self.odors0[trial] == carv and self.odors1[trial] == lim \
                                and self.flows0[trial] == carcon and self.flows1[trial] == limcon:
                                    
                                    
                                    concmat[i][j][1] += 1
                                    if self.rewards[trial] == 1:
                                        concmat[i][j][0] += 1
                                    if self.results[trial] > 4:
                                        concmat[i][j][3] +=1
                                        
                    for a in range(4):
                        for b in range(4):
                            if concmat[a][b][1]:
                                print concmat[a][b][0], concmat[a][b][1]
                                concmat[a][b][2] = 100.*float( concmat[a][b][0]) / float( concmat[a][b][1] - concmat[a][b][3])
                                print concmat[a][b][2], a, b, key, carvkey, limkey
                                #print concmat

                            else:
                                
                                concmat[a][b][2] = 0
                    

                    self.concmatdict[key][carvkey][limkey] = concmat
                    
                    # only taking the last calculated value of concmat[a][b][2]
                    # also only calculating values for the 4 comb of carvone+lim, not when either one equals None
                    print ''
        return self.concmatdict, matrix_list
                        
#print os.path.join( os.getcwd(), '8284_121416\\mouse8284_sess1_D2016_12_14T16_46_45.h5')    
#data = h5py.File( 'C:\VoyeurData\mouse_8283')
#day1 = Session(data)

def create_sessions():
    dirs = os.listdir('C:\VoyeurData\mouse_8283')
    session_instances = np.zeros((1, len(dirs)))
    for file in dirs:
        data = h5py.File(file)
        instance = Session(data)
        session_instances.append(instance)
    return session_instances
        
overall_dict = {}
#day1 = Session(h5py.File('C:\VoyeurData\mouse_8283\mouse8283_sess1_D2016_11_10T15_11_30.h5'))
#day2 = Session (h5py.File('C:\VoyeurData\mouse_8283\mouse8283_sess1_D2016_11_18T12_7_8.h5'))

def overall_performance(instances): # hits, misses, percent correct, etc from all trials of same odorset
    for instance in instances:
        for key in instance.keys():
            if key in overall_dict:
                    if key2 in overall_dict[keys]:
                        if key3 in overall_dict[key][key2]:
                            if mat in overalldict[key][key2][key3]:
                                overall_dict[key][key2][key3] += instance.key
                            else:
                                overall_dict[key][key2][key3] = np.zeros((4, 4, 3))
                        else:
                            overall_dict[key][key2] = instance.key3
                    else:
                        overall_dict[key] = instance.key2
            else: overall_dict = key
    return overall_dict

session_instances = create_sessions()
#session_instances = [day1, day2]
concmatdict, matrix_list = session_instances[1].make_concentration_bins()
print concmatdict
#print create_sessions()
#print overall_performance(create_sessions())

KeyError: "Unable to open object (Object 'trial0001' doesn't exist)"

In [ ]:
# create performance plots per day/session for each context

class Performance:

def __init__(self, data):
    
    
    
def calculate_pc(self): # instead of calculating points for each day, make a sliding window of ~100 trials
                        # and calculate performance for each window
                        # so maybe this doesn't need to be in a class since we'll want to pull out trials for each context
                        # and make a huge array of all the trials, then run a window over it and calculate performance
                        # for each window
    for day in days: # calculate point for each day
        for cont in contextlist:
            for i in numtrials: # find the trials that are in a certain context
                if context[i] == cont:
                    #calculate percent correct
                else:
                    pass

performance = []
sessioncounter = []

def create_performance_array(instance):
    for session in sessions:
        performance.append(instance.percentcorrect)
        sessioncounter.append([1]) # do this for each day, not each file?
        
    
# then take instances of each session and plot them vs number of sessions